#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [120]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [81]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\latip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Распакуем наши данные из архива. (нам изначально был выслан файл, поэтому этот шаг пропускаме)

In [ ]:
!unzip ...

Загрузим данные из csv файла в датафрейм.

In [121]:
df = pd.read_csv("input_data.csv", sep=';')

Отобразим на карте координаты наших построек.

In [125]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [108]:
print("Размер исходного датафрейма:", df.shape) #размер таблицы 
display(df.head(3)) #первые 3 строк таблицы 

print("\nПропуски по ключевым полям координат:")
print(df[['geo_lat','geo_lon']].isna().sum()) #количество пропусков в координатах 

Размер исходного датафрейма: (11358150, 15)


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0



Пропуски по ключевым полям координат:
geo_lat    0
geo_lon    0
dtype: int64


Мы будем вычислять от каждой точки в дата фрейм до центров Москвы и Санкт-Петербурга, вычисляя находятся ли они в пределах 20 км 
Задали широту и долготу Москвы и Питера, это будут примерные ориентиры 
Делаем мы это по формуле гаверсинуса, которая вычисляет расстояние между двумя точками на поверхности Земли если даны широта и долгота 

In [109]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # радисус Земли в км
    phi1 = np.radians(lat1) #переводим в радианы 
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1) #считаем разницу широт 
    dlambda = np.radians(lon2 - lon1) #разница долготы 
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2 #считаем а по формуле sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    #2R * arcsin(√a) возвращаем вот это значение в км длина дуги между точками 
    return 2 * R * np.arcsin(np.sqrt(a)) 

#ориентировочнеы координаты на которые ориентируемся 
MOSCOW_CENTER = (55.755864, 37.617698)
SPB_CENTER    = (59.938630, 30.314130)

#приводим все в флоат 
lat = df['geo_lat'].astype(float)
lon = df['geo_lon'].astype(float)

#считаем расстояние от каждой квартиры до центра москвы и питера 
df['dist_to_moscow'] = haversine_distance(lat, lon, MOSCOW_CENTER[0], MOSCOW_CENTER[1])
df['dist_to_spb']    = haversine_distance(lat, lon, SPB_CENTER[0], SPB_CENTER[1])

#признаки что если квартира к пределах - то 1, если не в приделах то - 0 
df['is_Moscow']            = (df['dist_to_moscow'] <= 20).astype(int)
df['is_Saint_Peterburg']   = (df['dist_to_spb']    <= 20).astype(int)
#выведем и проверим 
df[['geo_lat', 'geo_lon', 'dist_to_moscow', 'dist_to_spb', 'is_Moscow', 'is_Saint_Peterburg']].head(20)


,geo_lat,geo_lon,dist_to_moscow,dist_to_spb,is_Moscow,is_Saint_Peterburg
0,56.780112,60.699355,1422.965090,1790.088372,0,0
1,44.608154,40.138381,1252.277866,1826.452553,0,0
2,55.540060,37.725112,24.924791,657.033294,0,0
3,44.608154,40.138381,1252.277866,1826.452553,0,0
4,44.738685,37.713668,1225.073148,1760.723756,0,0
5,48.511172,44.566846,933.785411,1565.501300,0,0
6,55.009914,82.934859,2812.420554,3106.784695,0,0
7,51.834703,107.600571,4418.801050,4618.034596,0,0
8,45.003869,39.086511,1200.023668,1760.184191,0,0
9,53.164362,45.033956,558.861536,1170.784921,0,0


Если стоит 0 - то квартира НЕ в радиусе 20 км от Москвы или Питера,  
Если стоит 1 - то в радиусе 20 км 

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [111]:
#что вообще есть до 
print("Столбцы до удаления:")
print(df.columns.tolist())

#удалим ненужные признаки 
cols_to_drop = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 
                'street_id', 'id_region', 'house_id']

df = df.drop(columns=cols_to_drop, errors='ignore')

print("\nУдалены колонки:", cols_to_drop)

#выведем и посмотрим результат 
print("\nСтолбцы после удаления:")
print(df.columns.tolist())

#и посмотрим пару строк получтвшейся таблицы 
display(df.head())


Столбцы до удаления:
['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'dist_to_moscow', 'dist_to_spb', 'is_Moscow', 'is_Saint_Peterburg']

Удалены колонки: ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']

Столбцы после удаления:
['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'dist_to_moscow', 'dist_to_spb', 'is_Moscow', 'is_Saint_Peterburg']


,date,price,level,levels,rooms,area,kitchen_area,building_type,dist_to_moscow,dist_to_spb,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,1422.965090,1790.088372,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,1252.277866,1826.452553,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,24.924791,657.033294,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,1252.277866,1826.452553,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,1225.073148,1760.723756,0,0


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_Saint_Petersburg

Числовые: price, level, levels, rooms, area, kitchen_area, date

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Категориональнеы - те, которые сождержат тип данных, например класс или категорию, а не число для каких либо вычислений 
Числовые - которые содержат вечелину, с ними можно совершать арифмитические действия 

In [112]:
print(df.dtypes) 
#посмотрим какие типы колонок есть вообще 


date                   object
price                   int64
level                   int64
levels                  int64
rooms                   int64
area                  float64
kitchen_area          float64
building_type           int64
dist_to_moscow        float64
dist_to_spb           float64
is_Moscow               int64
is_Saint_Peterburg      int64
dtype: object



Дальше логически мы отправляет в категориальные - 'building_type', 'is_Moscow', 'is_Saint_Petersburg'
Остальные уйдут в числовые 

In [113]:
print(list(df.columns))


['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'dist_to_moscow', 'dist_to_spb', 'is_Moscow', 'is_Saint_Peterburg']


One-Hot Encoding — это техника преобразования категориальных переменных в бинарные векторы
в результате в таблицу добавились новые столбцы, представляющие различные категории, а исходные текстовые колонки мы удалили

In [114]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
#создаем объект энкодера 
encoder = OneHotEncoder(sparse_output=False, dtype=np.float32, handle_unknown='ignore') 
#будет флоат и игнорируем ошибки 

#обучаем энкодер на указанных колонках и сразу же преобразуем их в числовую матрицу
encoded = encoder.fit_transform(df[categorical_features])
#получаем имена новых колонок, которые создал энкодер
encoded_cols = encoder.get_feature_names_out(categorical_features) 
#превращаем резульатт обратно в дата фрейм
df_encoded = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)
#объединимиколонки по горизонтали 
df = pd.concat([df, df_encoded], axis=1)
#удалим исходные категориальные колонки
df = df.drop(columns=categorical_features) 
#смотрим результат
print(df.head())

         date     price  level  levels  rooms  area  kitchen_area  \
0  2021-01-01   2451300     15      31      1  30.3           0.0   
1  2021-01-01   1450000      5       5      1  33.0           6.0   
2  2021-01-01  10700000      4      13      3  85.0          12.0   
3  2021-01-01   3100000      3       5      3  82.0           9.0   
4  2021-01-01   2500000      2       3      1  30.0           9.0   

   dist_to_moscow  dist_to_spb  building_type_0  building_type_1  \
0     1422.965090  1790.088372              1.0              0.0   
1     1252.277866  1826.452553              1.0              0.0   
2       24.924791   657.033294              0.0              0.0   
3     1252.277866  1826.452553              1.0              0.0   
4     1225.073148  1760.723756              0.0              0.0   

   building_type_2  building_type_3  building_type_4  building_type_5  \
0              0.0              0.0              0.0              0.0   
1              0.0            

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [115]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
#превращаем строки в в формат datetime

first_date = df['date'].min()
# признак — количество дней с начала наблюдений, находит самую ранюю дату и считает сколько времени прошло
df['days_from_start'] = (df['date'] - first_date).dt.days

den = df['levels'].replace(0, np.nan)
df['level_ratio'] = (df['level'] / den).clip(lower=0, upper=1)
#признак отношения этажа к этажности - замена этажность, равную 0, на NaN то есть без деления на ноль
#делит этаж квартиры на общее количество этажей

#выводим новые признаки
print("Новые признаки добавлены:")
display(df[['date','days_from_start','level','levels','level_ratio']].head(8))

#удаляем дейт 
df = df.drop(columns=['date'])
print("Текущие столбцы:", list(df.columns)[:12], '...')

#маштабирование
#проверяет колонку прайс в случае да - сохраняет ее отдельно
target_col = 'price' if 'price' in df.columns else None
y = df[target_col].copy() if target_col else None

# определение числовых признаков 
#находим все числовые столбцы (int, float).
#убирает прайс чтобы не масштабировать её с признаками
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
if target_col and target_col in num_cols:
    num_cols.remove(target_col)

print("\nЧисловые признаки для скейлинга:", num_cols)

#пропуски заполним медианами 
df[num_cols] = df[num_cols].apply(lambda s: s.fillna(s.median()))

#маштабируем признаки - приводим все числовые признаки к одинаковому масштабу, это сам скейлер
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

print("\nСкейлинг готов. Выводы строк:")
display(df[num_cols].head())

#возвращает прайс если была
if target_col:
    df[target_col] = y

print("\nИтоговый размер dataframe:", df.shape)


Новые признаки добавлены:


,date,days_from_start,level,levels,level_ratio
0,2021-01-01,0.0,15,31,0.483871
1,2021-01-01,0.0,5,5,1.000000
2,2021-01-01,0.0,4,13,0.307692
3,2021-01-01,0.0,3,5,0.600000
4,2021-01-01,0.0,2,3,0.666667
5,2021-01-01,0.0,5,5,1.000000
6,2021-01-01,0.0,2,4,0.500000
7,2021-01-01,0.0,1,2,0.500000


Текущие столбцы: ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_moscow', 'dist_to_spb', 'building_type_0', 'building_type_1', 'building_type_2', 'building_type_3'] ...

Числовые признаки для скейлинга: ['level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_moscow', 'dist_to_spb', 'days_from_start', 'level_ratio']

Скейлинг готов. Выводы строк:


,level,levels,rooms,area,kitchen_area,dist_to_moscow,dist_to_spb,days_from_start,level_ratio
0,1.622769,2.665027,-0.621470,-0.840577,0.082486,0.156828,0.183501,-2.690957,-0.311150
1,-0.270043,-0.936859,-0.621470,-0.741051,0.267565,0.008616,0.215324,-2.690957,1.468814
2,-0.459324,0.171414,1.106234,1.175756,0.452644,-1.057124,-0.808065,-2.690957,-0.918734
3,-0.648605,-0.936859,1.106234,1.065171,0.360105,0.008616,0.215324,-2.690957,0.089342
4,-0.837886,-1.213927,-0.621470,-0.851636,0.360105,-0.015007,0.157803,-2.690957,0.319254



Итоговый размер dataframe: (11358150, 21)


В обшем и целом - добавили два новых числовых признака.
После этого все числовые признаки были нормализованы с помощью StandardScaler,
что привело их к общему масштабу (среднее 0, стандартное отклонение 1)

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

Пометочка для себя: 
у нас есть данные, когда приходит новый объект, мы ищем ему максимально похожих кентов по признакам и предсказываем его судьбу по средним признакам этих кентов 
И у нас х это будут признаки которые мы установили  а y - цены 

In [127]:
# целевая переменная — цена
y = df['price']

# признаки — всё, кроме цены
X = df.drop(columns=['price'])


In [128]:
print("Форма X:", X.shape)
print("Форма y:", y.shape)
print("Примеры признаков:")
display(X.head())


Форма X: (11358150, 14)
Форма y: (11358150,)
Примеры признаков:


,date,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


In [ ]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
#метож обучения модели
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self
#сохраним обучающие признаки (X) и целевое значение (y) в виде массивов
#метод предсказаний 
    def predict(self, X):
        X = np.array(X)
        y_pred = []

        #переберем все объекты, для которых хотим предсказать y
        for x in X:
            #расстояния от x до всех объектов обучения
            dists = distance.cdist([x], self.X_train, metric=self.metric)[0]

            #индексыk ближайших соседей
            knn_idx = np.argsort(dists)[:self.n_neighbors]

            #среднее начение y среди ближайших
            y_pred.append(np.mean(self.y_train[knn_idx]))

        return np.array(y_pred)


In [ ]:
#например
X_train = np.array([[1], [2], [3], [4], [5]])
y_train = np.array([2, 4, 6, 8, 10])  # например, цена растёт с признаком

model = KNNRegressor(n_neighbors=2)
model.fit(X_train, y_train)

#предскзание
X_test = np.array([[1.5], [3.5], [5.0]])
y_pred = model.predict(X_test)

print("Предсказания:", y_pred)


Предсказания: [3. 7. 9.]


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD',
                 epsilon=1e-8, decay_rate=0.9, max_iter=1000,
                 batch_size=None, random_state=None):
        # задаём параметры обучения
        self.learning_rate = learning_rate      #шаг обучения
        self.optimization  = optimization        #метод оптимизации
        self.epsilon       = epsilon             #чтобы не делит на ноль в адаград
        self.decay_rate    = decay_rate          #коэффициент для Momentum
        self.max_iter      = max_iter            # сколько итераций обучения
        self.batch_size    = batch_size          #размер мини-батча
        self.random_state  = random_state        # фиксируем рандом для воспроизводимости

        # инициализируем веса и биас
        self.weights = None
        self.bias    = None
        self.loss_history = []                  
     # чтобы хранить историю лосса

        # временные переменные для моментум и адаград
        self._v_w = None
        self._v_b = None
        self._g_w = None
        self._g_b = None

    @staticmethod
    def _mse(y_true, y_pred):
        # функция ошибки — среднеквадратичное отклонение
        return np.mean((y_true - y_pred) ** 2)

    def fit(self, X, y):
        # превращаем данные в нампай
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1)

        n_samples, n_features = X.shape
        rng = np.random.default_rng(self.random_state)

        # начальные веса и биас
        self.weights = np.zeros(n_features, dtype=float)
        self.bias    = 0.0

        # заготовки для моментум и адаград
        self._v_w = np.zeros_like(self.weights)
        self._v_b = 0.0
        self._g_w = np.zeros_like(self.weights)
        self._g_b = 0.0

        # основной цикл обучения
        for _ in range(self.max_iter):
            # если батч не указан — берём весь набор
            if self.batch_size is None or self.batch_size >= n_samples:
                Xb, yb = X, y
            else:
                # иначе берём случайный мини-батч
                idx = rng.choice(n_samples, size=self.batch_size, replace=False)
                Xb, yb = X[idx], y[idx]

            # прямой проход — предсказываем y
            y_pred = Xb @ self.weights + self.bias
            err = y_pred - yb

            # считаем градиент для весов и смещения
            m = len(yb)
            grad_w = (2.0/m) * (Xb.T @ err)
            grad_b = (2.0/m) * np.sum(err)

            # обновляем параметры в зависимости от метода оптимизации
            if self.optimization.lower() == 'sgd':
                # обычный стохастический градиентный спуск
                self.weights -= self.learning_rate * grad_w
                self.bias    -= self.learning_rate * grad_b

            elif self.optimization.lower() == 'momentum':
                # градиент с моментумом (ускорение по направлению предыдущего шага)
                beta = self.decay_rate
                self._v_w = beta * self._v_w + self.learning_rate * grad_w
                self._v_b = beta * self._v_b + self.learning_rate * grad_b
                self.weights -= self._v_w
                self.bias    -= self._v_b

            elif self.optimization.lower() == 'adagrad':
                # адаптивный шаг обучения который уменьшается при большои градиенте 
                self._g_w += grad_w**2
                self._g_b += grad_b**2
                self.weights -= (self.learning_rate / (np.sqrt(self._g_w) + self.epsilon)) * grad_w
                self.bias    -= (self.learning_rate / (np.sqrt(self._g_b) + self.epsilon)) * grad_b

            else:
                # если указано что-то левое — сразу ошибка
                raise ValueError("optimization must be 'SGD', 'Momentum', or 'AdaGrad'")

            # сохраняем ошибку на текущей итерации (на всех данных)
            self.loss_history.append(self._mse(y, self.predict(X)))

        return self

    def predict(self, X):
        # просто считаем y = Xw + b
        X = np.asarray(X, dtype=float)
        return X @ self.weights + self.bias


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [139]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


df = df.copy()  

#уюираем строковые дата колонки
if 'date' in df.columns:
    df = df.drop(columns=['date'])

#целевая переменная и признаки
y = df['price']
X = df.drop(columns=['price'])

# переводим всё в float
X = X.select_dtypes(include=['int64', 'float64']).astype(float)

# возьмём только часть данных, чтобы не перегружать память
df_small = df.sample(n=1000, random_state=42) 

#разделение на треен и тест 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Форма X_train: {X_train.shape}, X_test: {X_test.shape}")

#вспомогательная функция метрик 
def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

# ===============================================================
# 1️⃣ Твоя собственная модель KNN
# ===============================================================
my_knn = KNNRegressor(n_neighbors=5, metric='euclidean')
my_knn.fit(X_train.values, y_train.values)
y_pred_my_knn = my_knn.predict(X_test.values)
mse1, mae1, rmse1 = compute_metrics(y_test, y_pred_my_knn)

#наша линейная регрессия 
my_lr = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=300)
my_lr.fit(X_train.values, y_train.values)
y_pred_my_lr = my_lr.predict(X_test.values)
mse2, mae2, rmse2 = compute_metrics(y_test, y_pred_my_lr)


#модели из sklearn для сравнения

sk_knn = KNeighborsRegressor(n_neighbors=5)
sk_knn.fit(X_train, y_train)
y_pred_sk_knn = sk_knn.predict(X_test)
mse3, mae3, rmse3 = compute_metrics(y_test, y_pred_sk_knn)

sk_lr = LinearRegression()
sk_lr.fit(X_train, y_train)
y_pred_sk_lr = sk_lr.predict(X_test)
mse4, mae4, rmse4 = compute_metrics(y_test, y_pred_sk_lr)
#результаты
print("\nСравнение моделей:")
print(f"{'Модель':<25} {'MSE':>12} {'MAE':>12} {'RMSE':>12}")
print("-"*65)
print(f"{'Моя KNN':<25} {mse1:>12.4f} {mae1:>12.4f} {rmse1:>12.4f}")
print(f"{'Моя LinearRegression':<25} {mse2:>12.4f} {mae2:>12.4f} {rmse2:>12.4f}")
print(f"{'sklearn KNN':<25} {mse3:>12.4f} {mae3:>12.4f} {rmse3:>12.4f}")
print(f"{'sklearn LinearRegression':<25} {mse4:>12.4f} {mae4:>12.4f} {rmse4:>12.4f}")


Форма X_train: (9086520, 13), X_test: (2271630, 13)


KeyboardInterrupt: 